In [1]:
import csv
import requests
from bs4 import BeautifulSoup

In [2]:
FILENAME = '../data/election.csv'

In [3]:
domain = 'http://notelections.online/region'
link = '/region/st-petersburg?action=show&root=1&tvd=27820001217417&vrn=27820001217413&region=78&global=&sub_region=78&prver=0&pronetvd=null&vibid=27820001217417&type=222'

In [4]:
data = [['Тик', 'Уик', 'Количество избирателей', 'Полученные бюллетени', 'Выданные бюллетени в помещении',
         'Выданные бюллетени вне помещения', 'Погашенные бюллетени', 'Число бюллетеней в переносных',
         'Число бюллетеней в стационарных', 'Недействительные бюллетени', 'Действительные бюллетени',
         'Утраченные бюллетени', 'Неучтенные бюллетени', 'Амосов', 'Беглов', 'Тихонова']]

In [5]:
page = requests.get(domain + link)
soup = BeautifulSoup(page.text, 'html.parser')
go_reg = soup.find('select')
options = soup.find_all('option')[1:]
for i in range(len(options)):
    option = options[i]
    
    opt_soup = BeautifulSoup(requests.get(domain + option.get('value')).text, 'html.parser')
    
    tik = opt_soup.find_all('tr')[32:48]
    tik.pop(-4)
    
    data_uiks_of_tik = [[int(c.text.split(' ')[-1][1:]) for c in tik[0].find_all('a')]]
    tik.pop(0)
    
    for line in tik:
        data_uiks_of_tik.append([int(c.text.split('\n')[0]) for c in line.find_all('td')])
        
    tik_num = option.text.split(' ')[0]

    for j in range(len(data_uiks_of_tik[0])):
        row = [tik_num]
        for k in range(15):
            row.append(data_uiks_of_tik[k][j])
        data.append(row)

In [6]:
with open(FILENAME, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerows(data)